In [1]:
import pandas as pd
import numpy as np
import re
import csv
import time
from datetime import datetime, timedelta
import snscrape.modules.twitter as sntwitter

In [2]:
def createQuery(word, start, end):
    return word + " (lang:en) since:" + start + " until:" + end

In [3]:
words = ["russia",
        "ukraine",
        "putin",
        "zelensky",
        "war"]

start = datetime(2022, 1, 1)
end = datetime(2023, 4, 30)

output_format = "%Y-%m-%d"

tweets = []
wordLimitGlobal = 500

col_names =  ['Date', 'Tweet Treated', 'Tweet Raw', 'Url', 'Id']

In [4]:
duration = end - start
duration = int(duration.days)

startAux = start

In [5]:
errors = []

In [ ]:
wordLimit = wordLimitGlobal

for i in range(1, duration+1):
    endString = startAux + timedelta(days=1)
    endString = endString.strftime(output_format)

    try: 
        for w in words:
            query = createQuery(w, startAux.strftime(output_format), endString)
            print("QUERY: " + query)

            cnt = 0
            for tweet in sntwitter.TwitterSearchScraper(query).get_items():
                if cnt == wordLimit:
                    break
                else:   
                    no_link = re.sub(r'https?://\S+', '', tweet.rawContent.lower())
                    no_link_no_user = re.sub(r'\@[\w\_]+', '', no_link)

                    tweets.append([tweet.date, no_link_no_user, tweet.rawContent, tweet.url, tweet.id])
                    cnt += 1
    except:
        errors.append(startAux)
        time.sleep(120)

    dfTemp = pd.DataFrame(tweets, columns=col_names)
    dfTemp = pd.DataFrame(dfTemp.drop_duplicates(subset=['Id'], keep='last'))
    dfTemp.to_csv("data/"+startAux.strftime(output_format)+".csv", index=False, quoting=csv.QUOTE_NONNUMERIC)

    tweets = []
    startAux = startAux + timedelta(days=1)

In [7]:
dfErrors = pd.DataFrame(errors)
dfErrors.to_csv("errors.csv", index=False, quoting=csv.QUOTE_NONNUMERIC)

In [ ]:
import os

dfAll = pd.DataFrame(columns=col_names)


path = "data/"
with os.scandir(path) as entries:
    for entry in entries:
        dfRunning = pd.read_csv("data/"+str(entry.name))
        dfAll = dfAll.append(dfRunning)

In [17]:
dfAll

,Date,Tweet Treated,Tweet Raw,Url,Id
0,2022-01-01 23:59:41+00:00,goldfinger &amp; from russia with love,@donwinslow Goldfinger &amp; From Russia with ...,https://twitter.com/coolvee2222/status/1477429...,1477429351952130051
1,2022-01-01 23:59:23+00:00,thank you! russia musume 1 or dream note please!,@aiuncensored thank you! russia musume 1 or dr...,https://twitter.com/CsarVsq30904305/status/147...,1477429275926179841
2,2022-01-01 23:59:23+00:00,"new in shop! vintage russian nesting dolls, cl...","New in Shop! Vintage Russian Nesting Dolls, Cl...",https://twitter.com/trashtique/status/14774292...,1477429275024310273
3,2022-01-01 23:59:13+00:00,if the chinese government donated to the gqp ...,@RepThomasMassie If the Chinese government don...,https://twitter.com/SouthernNotSt/status/14774...,1477429235589566466
4,2022-01-01 23:58:55+00:00,i'm dying to know how redacted tonight fits i...,@jimstewartson I'm dying to know how Redacted ...,https://twitter.com/rscobe1920/status/14774291...,1477429159307661317
...,...,...,...,...,...
2240,2023-04-29 23:51:58+00:00,losing the idgaf war too..,losing the idgaf war too..,https://twitter.com/calicoception/status/16524...,1652460747606093824
2241,2023-04-29 23:51:55+00:00,"zelensky has already lost this war,\n\nno way...",@Spriter99880 Zelensky has already lost this w...,https://twitter.com/JefferyLivesay/status/1652...,1652460736839393280
2242,2023-04-29 23:51:54+00:00,war thunder?,@ImperialStates War thunder?,https://twitter.com/MarkovicNoor/status/165246...,1652460734352089090
2243,2023-04-29 23:51:54+00:00,recruitment in arm forces has been in trouble...,@RonnyJacksonTX Recruitment in arm forces has ...,https://twitter.com/alex_duitsman/status/16524...,1652460732439724032


In [18]:
pd.DataFrame(dfAll.drop_duplicates(subset=['Id'], keep='last'))

,Date,Tweet Treated,Tweet Raw,Url,Id
0,2022-01-01 23:59:41+00:00,goldfinger &amp; from russia with love,@donwinslow Goldfinger &amp; From Russia with ...,https://twitter.com/coolvee2222/status/1477429...,1477429351952130051
1,2022-01-01 23:59:23+00:00,thank you! russia musume 1 or dream note please!,@aiuncensored thank you! russia musume 1 or dr...,https://twitter.com/CsarVsq30904305/status/147...,1477429275926179841
2,2022-01-01 23:59:23+00:00,"new in shop! vintage russian nesting dolls, cl...","New in Shop! Vintage Russian Nesting Dolls, Cl...",https://twitter.com/trashtique/status/14774292...,1477429275024310273
3,2022-01-01 23:59:13+00:00,if the chinese government donated to the gqp ...,@RepThomasMassie If the Chinese government don...,https://twitter.com/SouthernNotSt/status/14774...,1477429235589566466
4,2022-01-01 23:58:55+00:00,i'm dying to know how redacted tonight fits i...,@jimstewartson I'm dying to know how Redacted ...,https://twitter.com/rscobe1920/status/14774291...,1477429159307661317
...,...,...,...,...,...
2240,2023-04-29 23:51:58+00:00,losing the idgaf war too..,losing the idgaf war too..,https://twitter.com/calicoception/status/16524...,1652460747606093824
2241,2023-04-29 23:51:55+00:00,"zelensky has already lost this war,\n\nno way...",@Spriter99880 Zelensky has already lost this w...,https://twitter.com/JefferyLivesay/status/1652...,1652460736839393280
2242,2023-04-29 23:51:54+00:00,war thunder?,@ImperialStates War thunder?,https://twitter.com/MarkovicNoor/status/165246...,1652460734352089090
2243,2023-04-29 23:51:54+00:00,recruitment in arm forces has been in trouble...,@RonnyJacksonTX Recruitment in arm forces has ...,https://twitter.com/alex_duitsman/status/16524...,1652460732439724032


In [19]:
dfAll.to_csv("completeDataset.csv", index=False, quoting=csv.QUOTE_NONNUMERIC)

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("facebook/roberta-hate-speech-dynabench-r1-target")
model = AutoModelForSequenceClassification.from_pretrained("facebook/roberta-hate-speech-dynabench-r1-target")

ValueError: Unable to compare versions for numpy>=1.17: need=1.17 found=None. This is unusual. Consider reinstalling numpy.

In [ ]:
import torch

device = "cuda:0" if torch.cuda.is_available() else "cpu"